In [1]:
import skimage
import os
from skimage import io
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.color import rgb2hsv
import re
import numpy as np
import seaborn as sns
## for the stats class
import scipy.signal as sg
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.decomposition import SparsePCA
import pywt
from scipy import stats
import timeit 
#import optshrink as opt # package we create
import numpy as np
# import scipy.io as sio
import h5py
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import seaborn as sns
import pandas as pd
from datetime import datetime
import pytz
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm
from skimage import io
from patchify import patchify, unpatchify



import collections
# from itertools import chain
# import urllib.request as request
# import pickle 

import numpy as np

import scipy.signal as signal
import scipy.special as special
import scipy.optimize as optimize

import matplotlib.pyplot as plt

import skimage.io
import skimage.transform

# import cv2

# from libsvm import svmutil

import Shady.Contrast as sc 

In [2]:

# https://github.com/ocampor/notebooks/blob/master/notebooks/image/quality/brisque.ipynb
def normalize_kernel(kernel):
    return kernel / np.sum(kernel)

def gaussian_kernel2d(n, sigma):
    Y, X = np.indices((n, n)) - int(n/2)
    gaussian_kernel = 1 / (2 * np.pi * sigma ** 2) * np.exp(-(X ** 2 + Y ** 2) / (2 * sigma ** 2)) 
    return normalize_kernel(gaussian_kernel)

def local_mean(image, kernel):
    return signal.convolve2d(image, kernel, 'same')


def local_deviation(image, local_mean, kernel):
    "Vectorized approximation of local deviation"
    sigma = image ** 2
    sigma = signal.convolve2d(sigma, kernel, 'same')
    return np.sqrt(np.abs(local_mean ** 2 - sigma))


def calculate_mscn_coefficients(image, kernel_size=6, sigma=7/6):
    C = 1/255
    kernel = gaussian_kernel2d(kernel_size, sigma=sigma)
    local_mean = signal.convolve2d(image, kernel, 'same')
    local_var = local_deviation(image, local_mean, kernel)
    
    return (image - local_mean) / (local_var + C)

def calculate_pair_product_coefficients(mscn_coefficients):
    return collections.OrderedDict({
        'mscn': mscn_coefficients,
        'horizontal': mscn_coefficients[:, :-1] * mscn_coefficients[:, 1:],
        'vertical': mscn_coefficients[:-1, :] * mscn_coefficients[1:, :],
        'main_diagonal': mscn_coefficients[:-1, :-1] * mscn_coefficients[1:, 1:],
        'secondary_diagonal': mscn_coefficients[1:, :-1] * mscn_coefficients[:-1, 1:]
    })

In [3]:
image_dir = '/home/smmrrr/surfline/'

image_files = os.listdir(image_dir) 
print(len(image_files))

photo_links = pd.DataFrame({'photo' : image_files})
# photo_info = pd.DataFrame({'project_files_location' : project_files, 'photo_size' : size})
# photo_info.to_csv('/home/smmrrr/Fog_Imaging_Project/all_photos_link.csv', index = False)
photo_links=photo_links[photo_links['photo'].str.contains('jpg')]

42399


In [14]:
photo_labels = pd.read_csv('/home/smmrrr/Fog_Imaging_Project/sta_221/surfline_photo_labels.csv')[[ 'Label', 'photo']]
photo_labels.loc[photo_labels['Label'] != 'uncertain']

# # photo_links
links_and_labels = photo_links.merge(photo_labels, how = 'left'
                        , on = 'photo')
links_and_labels
image_files_summary = links_and_labels['photo'].str.split(r'[\.|\_|\-]',expand = True)
image_files_summary.columns = ['year', 'month', 'day', 'hour_central','site', 'ext', 't', 'tt']
image_files_summary['ext'].unique()
links_and_labels = pd.concat([links_and_labels, image_files_summary], axis = 1 )
# links_and_labels = links_and_labels.loc[links_and_labels['ext'] =='jpg']
links_and_labels['year'] = links_and_labels['year'].astype(int)
links_and_labels['day']=links_and_labels['day'].astype(int)
links_and_labels['hour_central']=links_and_labels['hour_central'].astype(int)
links_and_labels['month']=links_and_labels['month'].astype(int)
links_and_labels['hour_central']=np.round(links_and_labels['hour_central']/100).astype(int) 
links_and_labels['time'] = links_and_labels.apply(lambda row: datetime(row['year'], row['month'], row['day'], row['hour_central']), axis=1)

# # Specify the original timezone (if different from system timezone)
original_timezone = pytz.timezone('US/Central')

# # Convert to Pacific Time
pacific_timezone = pytz.timezone('US/Pacific')
links_and_labels['time_pst'] = links_and_labels['time'].dt.tz_localize(original_timezone).dt.tz_convert(pacific_timezone)
links_and_labels['hour_pst'] = links_and_labels['time_pst'].dt.hour
links_and_labels['Label'] = links_and_labels['Label'].str.lower()

links_and_labels=links_and_labels.drop([ 't', 'tt', 'ext'], axis = 1)

links_and_labels['site'] = links_and_labels['site'].str.replace(' 11', '')

links_and_labels['Label'].fillna('not labeled', inplace=True)

links_and_labels

,photo,Label,year,month,day,hour_central,site,time,time_pst,hour_pst
0,2023_07-18.0730_lincolncityor.jpg,not labeled,2023,7,18,7,lincolncityor,2023-07-18 07:00:00,2023-07-18 05:00:00-07:00,5
1,2023_05-09.0800_lincolncityor.jpg,not labeled,2023,5,9,8,lincolncityor,2023-05-09 08:00:00,2023-05-09 06:00:00-07:00,6
2,2023_06-10.1000_otterrockor.jpg,not labeled,2023,6,10,10,otterrockor,2023-06-10 10:00:00,2023-06-10 08:00:00-07:00,8
3,2022_08-17.1230_otterrockor.jpg,not labeled,2022,8,17,12,otterrockor,2022-08-17 12:00:00,2022-08-17 10:00:00-07:00,10
4,2023_07-12.2000_lincolncityor.jpg,not labeled,2023,7,12,20,lincolncityor,2023-07-12 20:00:00,2023-07-12 18:00:00-07:00,18
...,...,...,...,...,...,...,...,...,...,...
42392,2023_06-09.1000_lincolncityor.jpg,foggy,2023,6,9,10,lincolncityor,2023-06-09 10:00:00,2023-06-09 08:00:00-07:00,8
42393,2023_08-20.0030_otterrockor.jpg,not labeled,2023,8,20,0,otterrockor,2023-08-20 00:00:00,2023-08-19 22:00:00-07:00,22
42394,2023_06-28.1330_otterrockor.jpg,not labeled,2023,6,28,13,otterrockor,2023-06-28 13:00:00,2023-06-28 11:00:00-07:00,11
42395,2022_09-08.0900_lincolncityor.jpg,not_foggy,2022,9,8,9,lincolncityor,2022-09-08 09:00:00,2022-09-08 07:00:00-07:00,7


In [15]:
links_and_labels['hour_pst'].value_counts()

5     2398
6     2391
7     2388
11    2370
8     2369
10    2368
9     2366
12    2364
13    2360
14    2356
16    2354
15    2354
18    2354
17    2354
19    2335
20    2322
21    2309
22    2285
Name: hour_pst, dtype: int64

In [16]:
links_and_labels.loc[links_and_labels['Label'].isin(['foggy', 'not_foggy']),'hour_pst'].value_counts()

8     624
13    619
11    615
18    614
12    613
7     607
17    605
16    602
10    599
9     595
15    590
14    560
19    558
6     229
20    213
21    175
5     170
22     41
Name: hour_pst, dtype: int64

In [17]:
links_and_labels = links_and_labels.loc[links_and_labels['hour_pst'].between(7, 19)]

In [28]:
links_and_labels.loc[links_and_labels['Label'].isin(['foggy', 'not_foggy']),['hour_pst', 'Label']].value_counts(
).reset_index(
).pivot(index = 'hour_pst', columns = 'Label' )

0          
Label    foggy not_foggy
hour_pst                
7          199       408
8          181       443
9          136       459
10         103       496
11         101       514
12          88       525
13          97       522
14          88       472
15          94       496
16         102       500
17         110       495
18         102       512
19         101       457

In [29]:
links_and_labels.to_csv('/home/smmrrr/Fog_Imaging_Project/Fog_Project_Code/desc_photos_for_model.csv'
,index=False)